## Practice running the Permafrost Discovery Gateway visualization workflow through web tiling

- 3 adjacent ice wedge polygon files from [Wrangle Island](https://www.google.com/maps/place/Wrangel+Island/@71.3497058,179.8238705,9z/data=!4m6!3m5!1s0x50a70636a5f5033f:0xe1dca925085b4bc3!8m2!3d71.2488724!4d-179.9789208!16zL20vMDMyZnRq), off the coasts of Russia & Alaska
- use environment with `viz-staging` and `viz-raster` installed
- after running through these steps in chunks in this notebook, it's a great idea to transfer the code to a script and run as a `tmux` session

In [ ]:
# input data import
from pathlib import Path

# staging
import pdgstaging
from pdgstaging import TileStager

# rasterization
import pdgraster
from pdgraster import RasterTiler

# 3D tiling
from viz_3dtiles import TreeGenerator, BoundingVolumeRegion

# visual checks
import geopandas as gpd

# logging
from datetime import datetime
import logging
import logging.handlers
import os

### Import data

In order to process 1 or 2 files instead of all 3 adjacent files on Wrangle Island, subset the `input` list of filepaths created below. Estimated times and number of files 

In [ ]:
base_dir = Path('/home/jcohen/iwp_russia_subset_clipToFP_PR/iwp')
filename = '*.shp'
# To define each .shp file within each subdir as a string representation with forward slashes, use as_posix()
# The ** represents that any subdir string can be present between the base_dir and the filename
input = [p.as_posix() for p in base_dir.glob('**/' + filename)]
input

In [ ]:
# pull filepaths for footprints in the same way we pulled IWP shp file paths
base_dir_fp = Path('/home/jcohen/iwp_russia_subset_clipToFP_PR/footprints')
# To define each .shp file within each subdir as a string representation with forward slashes, use as_posix()
# The ** represents that any subdir string can be present between the base_dir and the filename
fps = [p.as_posix() for p in base_dir_fp.glob('**/' + filename)]
fps

### Define the logging configuration

This prints logging statements to a file specified by the path in the config. Change the filepath as needed. There will be many logging statements written to that file. It's helpful to ctrl + f for certain logged statements when troubleshooting. For example, if fewer files were staged than expected, you can search for "error" or "failed". If you are debugging a silent error and suspect that the issue has something to do with the order in which input files are processed, you can search for the input filenames to determine which was staged first. In between runs, it's a good idea to delete the log from the past run, rename it, or archive it elsewhere so the next run's log does not append to the same log file. 

In [ ]:
handler = logging.handlers.WatchedFileHandler(
    os.environ.get("LOGFILE", "/home/jcohen/pdg-info/helpful-code/log.log"))
formatter = logging.Formatter(logging.BASIC_FORMAT)
handler.setFormatter(formatter)
root = logging.getLogger()
root.setLevel(os.environ.get("LOGLEVEL", "INFO"))
root.addHandler(handler)

#### Plot the files with their footprints (optional)

Each of the following plots takes ~2-5 minutes to generate. Size has been increased in order to see how the IWP polygons spill over the edge of the footprints.

In [ ]:
# first pair
shp = gpd.read_file(input[0])
footprint = gpd.read_file(fps[0])
ax = shp.plot(color='none', edgecolor='green', linewidths=1.5, figsize=(14,14))
footprint.plot(ax=ax, color='none', edgecolor='black', linewidths=0.5, figsize=(14,14))

In [ ]:
# second pair
shp = gpd.read_file(input[1])
footprint = gpd.read_file(fps[1])
ax = shp.plot(color='none', edgecolor='blue', linewidths=1.5, figsize=(14,14))
footprint.plot(ax=ax, color='none', edgecolor='black', linewidths=0.5, figsize=(14,14))

In [ ]:
# third pair
shp = gpd.read_file(input[2])
footprint = gpd.read_file(fps[2])
ax = shp.plot(color='none', edgecolor='green', linewidths=1.5, figsize=(14,14))
footprint.plot(ax=ax, color='none', edgecolor='black', linewidths=0.5, figsize=(14,14))

In [ ]:
# plot 2 non-coastal adjacent iwp files together, no footprints
shp1 = gpd.read_file(input[1])
shp2 = gpd.read_file(input[2])
ax = shp1.plot(color='none', edgecolor='blue', linewidths=1.5, figsize=(14,14), alpha=0.3)
shp2.plot(ax=ax, color='none', edgecolor='green', linewidths=0.5, figsize=(14,14), alpha=0.3)

### Define the stager

You can input the config directly to `TileStager()`, or save the config as an object earlier in the script, or save the config as a separate `.py` or `.json` script and source it in with:
```python
# config.py must be a script in same level of folder hierachy as this notebook 
# that contains config defined as an object like `config = {...}` 
import config 
config = config.config
pdgstaging.TileStager(config)
```

- Many of the settings in the config specified below are defaults. You can find default values for the config [here](https://github.com/PermafrostDiscoveryGateway/viz-staging/blob/4f31e951600d54c128f76b48a47ec390261fb548/pdgstaging/ConfigManager.py#L351-L422).
- You'll notice that the statistics (calculated during rasterization, each becomes a band) are tailored towards ice wedge polygons. You can remove an entire statistic, or change them as desired. The "name" of each statistic can be anything.
- I would not recommend changing the number of z-levels. 
- Color palette can be changed, but must be at least 2 values.
- Input data types for the staging step should be vectors, like `.shp` or `.gpkg`. We specify `.shp` below because the IWP input data are shapefiles. 

For more information about the configuration, there is extensive documentation in [PermafrostDiscoveryGateway/viz-staging/pdgstaging/ConfigManager.py](https://github.com/PermafrostDiscoveryGateway/viz-staging/blob/main/pdgstaging/ConfigManager.py)


In [ ]:
stager = TileStager({
  "deduplicate_clip_to_footprint": True, 
  "dir_input": "/home/jcohen/iwp_russia_subset_clipToFP_PR/iwp/", 
  "ext_input": ".shp",
  "ext_footprints": ".shp",
  "dir_footprints": "/home/jcohen/iwp_russia_subset_clipToFP_PR/footprints/", 
  "dir_staged": "staged/",
  "dir_geotiff": "geotiff/", 
  "dir_web_tiles": "web_tiles/", 
  "filename_staging_summary": "staging_summary.csv",
  "filename_rasterization_events": "raster_events.csv",
  "filename_rasters_summary": "raster_summary.csv",
  "filename_config": "config",
  "simplify_tolerance": 0.1,
  "tms_id": "WGS1984Quad",
  "z_range": [
    0,
    15
  ],
  "geometricError": 57,
  "z_coord": 0,
  "statistics": [
    {
      "name": "iwp_coverage",
      "weight_by": "area",
      "property": "area_per_pixel_area",
      "aggregation_method": "sum",
      "resampling_method": "average",
      "val_range": [
        0,
        1
      ],
      "palette": [
        "#66339952",
        "#ffcc00"
      ],
      "nodata_val": 0,
      "nodata_color": "#ffffff00"
    },
  ],
  "deduplicate_at": [
    "raster",
    "3dtile"
  ],
  "deduplicate_keep_rules": [
    [
      "Date",
      "larger"
    ]
  ],
  "deduplicate_method": "footprints"
})

In [ ]:
stager.stage_all()
# took about 26 minutes for all 3 files to stage

## Rasterization & web-tiling

- Use the `pdgraster.RasterTiler()` function to create the `rasterizer`, then use it to execute the rasterize function: `rasterizer.rasterize_vectors()`.
- Alternatively, can run these two steps as one with just `pdgraster.RasterTiler().rasterize_all()`. Just as in staging, you can input the config directly to `RasterTiler()`, or save the config as an object earlier in the script, or save the config as a separate script and source it in. 
- `rasterize_all()` is a wrapper for `rasterize_vectors()`. It pulls all staged filepaths from the staged dir and rasterizes all z-levels . We do _not_ use `rasterize_all()` when rasterizing in parallel with `parsl` or `ray`. It is exclusively used for small datasets that are not run in parallel. We have to create specific `@parsl` or `@ray.remote` wrapper functions around `stage()` and `rasterize_vectors()` if using those packages for staging, rasterizaiton, etc. For an example, see [here](https://github.com/PermafrostDiscoveryGateway/viz-workflow/blob/8c1997a9d2456bcb79ba1b3ab0f82b3b2b30b141/IN_PROGRESS_VIZ_WORKFLOW.py#L668-L693) for how we rasterize in the ray workflow.
- Rasterization when executed with `rasterize_all()` creates `.tif` files in the output `geotiff` dir, _and_ creates the same number of `.png` files in the output `web_tiles` dir.
    - When we use `rasterize_vectors()` instead, we _only create the `.tif` files and not the `.png` files_. So that needs to be executed as a separate step with `rasterizer.webtiles_from_geotiffs()` after the `rasterize_vectors() step`, and _in between those steps_ we need to manually "update the ranges" in the rasterizer to ensure that the colors within one z-level of `.png` files looks appropriate when visualized in the portal. We will cross that bridge as necessay, just as we do [here](https://github.com/PermafrostDiscoveryGateway/viz-workflow/blob/8c1997a9d2456bcb79ba1b3ab0f82b3b2b30b141/IN_PROGRESS_VIZ_WORKFLOW.py#L592) in the `ray` workflow on Delta.
- The number of z-level 15 tiles in the `staged` dir should match the number of z-level 15 tiles in the `geotiff` and `web_tiles` dirs. The total number of files in both the `geotiff` and `web_tiles` dirs is a _lot_ more than the number of files in `staged` because `staged` _only contains the highest zoom level_ with no parent z-levels.
- The web tiles are what we actually visualize on the PDG portal and local cesium. We create the rasters for summary stats (the data behind the web tiles, stored in bands of the `.tif` files)




In [ ]:
RasterTiler({
  "deduplicate_clip_to_footprint": True, 
  "dir_input": "/home/jcohen/iwp_russia_subset_clipToFP_PR/iwp/", 
  "ext_input": ".shp",
  "ext_footprints": ".shp",
  "dir_footprints": "/home/jcohen/iwp_russia_subset_clipToFP_PR/footprints/", 
  "dir_staged": "staged/",
  "dir_geotiff": "geotiff/", 
  "dir_web_tiles": "web_tiles/", 
  "filename_staging_summary": "staging_summary.csv",
  "filename_rasterization_events": "raster_events.csv",
  "filename_rasters_summary": "raster_summary.csv",
  "filename_config": "config",
  "simplify_tolerance": 0.1,
  "tms_id": "WGS1984Quad",
  "z_range": [
    0,
    15
  ],
  "geometricError": 57,
  "z_coord": 0,
  "statistics": [
    {
      "name": "iwp_coverage",
      "weight_by": "area",
      "property": "area_per_pixel_area",
      "aggregation_method": "sum",
      "resampling_method": "average",
      "val_range": [
        0,
        1
      ],
      "palette": [
        "#66339952",
        "#ffcc00"
      ],
      "nodata_val": 0,
      "nodata_color": "#ffffff00"
    },
  ],
  "deduplicate_at": [
    "raster"
  ],
  "deduplicate_keep_rules": [
    [
      "Date",
      "larger"
    ]
  ],
  "deduplicate_method": "footprints"
}).rasterize_all()

# took about 61 min for all staged files to rasterize and web tile

Number of `geotiff` files produced (for all z-levels): **4376**

Number of `web_tiles` produced (for all z-levels): **4376**

-----------

You can use `rasterio` to plot the rasters if you want to get an idea what the statistics look like. Change the number to choose which band (statistic) to visualize. Example:

```python
import matplotlib.pyplot as plt
import rasterio

file1 = rasterio.open('/path/to/raster.tif')

plt.imshow(file1.read(1), cmap='pink')
```

## Visualizing the web tiles

Ypu can simply open the `.png` files to view them one at a time, but it's much better to view them on a Cesium basemap! For steps for how to visualize the web tiles with local Cesium, see [documentation here in pdg-info](https://github.com/robyngit/pdg-info/blob/main/05_displaying-the-tiles.md#option-1-run-cesium-locally).

## 3D tiling

This step pulls in vector data from the `staged` dir and processes it into 2 file formats: `.b3dm` files and `.json` files that accompany the `.b3dm` files only if they are the highest z-level. This uses the same config as staging and rasterization.

When the workflow meets the following criteria:
1. It is intended to run through all steps staging, rasterization/web tiling, and 3dtiling
2. It is set to deduplicate in the config (rather than setting to `None`),
3.  The staged data is intended to be archived and available for others to download and use

...then the config is usually set to deduplicate at both `raster` and `3dtiles` rather than `staging`. This is because we want the "raw" staging data to be un-deduplicated and available for use when it is archived. See [here](https://github.com/PermafrostDiscoveryGateway/viz-staging/blob/4f31e951600d54c128f76b48a47ec390261fb548/pdgstaging/ConfigManager.py#L231-L294) in the `ConfigManager.py` for this documentation. For data files that do not have accompnaying "footprints" like the ice wedge polygon data, the deduplication method would be "neighbor". This is the case for DRP data.

For 3d-tiling, we need to "manually" import `StagedTo3DConverter` (rather than sourcing it in).

In [ ]:
class StagedTo3DConverter():
    """
        Processes staged vector data into Cesium 3D tiles according to the
        settings in a config file or dict. This class acts as the orchestrator
        of the other viz-3dtiles classes, and coordinates the sending and
        receiving of information between them.
    """

    def __init__(
        self,
        config
    ):
        """
            Initialize the StagedTo3DConverter class.

            Parameters
            ----------

            config : dict or str
                A dictionary of configuration settings or a path to a config
                JSON file. (See help(pdgstaging.ConfigManager))
        """

        self.config = pdgstaging.ConfigManager(config)
        self.tiles = pdgstaging.TilePathManager(
            **self.config.get_path_manager_config())

    def all_staged_to_3dtiles(
        self
    ):
        """
            Process all staged vector tiles into 3D tiles.
        """

        # Get the list of staged vector tiles
        paths = self.tiles.get_filenames_from_dir('staged')
        # Process each tile
        for path in paths:
            self.staged_to_3dtile(path)

    def staged_to_3dtile(self, path):
        """
            Convert a staged vector tile into a B3DM tile file and a matching
            JSON tileset file.

            Parameters
            ----------
            path : str
                The path to the staged vector tile.

            Returns
            -------
            tile, tileset : Cesium3DTile, Tileset
                The Cesium3DTiles and Cesium3DTileset objects
        """

        try:

            # Get information about the tile from the path
            tile = self.tiles.tile_from_path(path)
            out_path = self.tiles.path_from_tile(tile, '3dtiles')

            tile_bv = self.bounding_region_for_tile(tile)

            # Get the filename of the tile WITHOUT the extension
            tile_filename = os.path.splitext(os.path.basename(out_path))[0]
            # Get the base of the path, without the filename
            tile_dir = os.path.dirname(out_path) + os.path.sep

            # Log the event
            logger.info(
                f'Creating 3dtile from {path} for tile {tile} to {out_path}.')

            # Read in the staged vector tile
            gdf = gpd.read_file(path)

            # Check if the gdf is empty
            if len(gdf) == 0:
                logger.warning(
                    f'Vector tile {path} is empty. 3D tile will not be'
                    ' created.')
                return

            # Remove polygons with centroids that are outside the tile boundary
            prop_cent_in_tile = self.config.polygon_prop(
                'centroid_within_tile')
            gdf = gdf[gdf[prop_cent_in_tile]]

            # Check if deduplication should be performed
            dedup_here = self.config.deduplicate_at('3dtiles')
            dedup_method = self.config.get_deduplication_method()

            # Deduplicate if required
            if dedup_here and (dedup_method is not None):
                dedup_config = self.config.get_deduplication_config(gdf)
                dedup = dedup_method(gdf, **dedup_config)
                gdf = dedup['keep']

                # The tile could theoretically be empty after deduplication
                if len(gdf) == 0:
                    logger.warning(
                        f'Vector tile {path} is empty after deduplication.'
                        ' 3D Tile will not be created.')
                    return

            # Create & save the b3dm file
            ces_tile, ces_tileset = TreeGenerator.leaf_tile_from_gdf(
                gdf,
                dir=tile_dir,
                filename=tile_filename,
                z=self.config.get('z_coord'),
                geometricError=self.config.get('geometricError'),
                tilesetVersion=self.config.get('version'),
                boundingVolume=tile_bv
            )

            return ces_tile, ces_tileset

        except Exception as e:
            logger.error(f'Error creating 3D Tile from {path}.')
            logger.error(e)

    def parent_3dtiles_from_children(self, tiles, bv_limit=None):
        """
            Create parent Cesium 3D Tileset json files that point to
            of child JSON files in the tile tree hierarchy.

            Parameters
            ----------
            tiles : list of morecantile.Tile
                The list of tiles to create parent tiles for.
        """

        tile_manager = self.tiles
        config_manager = self.config

        tileset_objs = []

        # Make the next level of parent tiles
        for parent_tile in tiles:
            # Get the path to the parent tile
            parent_path = tile_manager.path_from_tile(parent_tile, '3dtiles')
            # Get just the base dir without the filename
            parent_dir = os.path.dirname(parent_path)
            # Get the filename of the parent tile, without the extension
            parent_filename = os.path.basename(parent_path)
            parent_filename = os.path.splitext(parent_filename)[0]
            # Get the children paths for this parent tile
            child_paths = tile_manager.get_child_paths(parent_tile, '3dtiles')
            # Remove paths that do not exist
            child_paths = tile_manager.remove_nonexistent_paths(child_paths)
            # Get the parent bounding volume
            parent_bv = self.bounding_region_for_tile(
                parent_tile, limit_to=bv_limit)
            # If the bounding region is outside t
            # Get the version
            version = config_manager.get('version')
            # Get the geometric error
            geometric_error = config_manager.get('geometricError')
            # Create the parent tile
            tileset_obj = TreeGenerator.parent_tile_from_children_json(
                child_paths,
                dir=parent_dir,
                filename=parent_filename,
                geometricError=geometric_error,
                tilesetVersion=version,
                boundingVolume=parent_bv
            )
            tileset_objs.append(tileset_obj)

        return tileset_objs

    def bounding_region_for_tile(self, tile, limit_to=None):
        """
        For a morecantile.Tile object, return a BoundingVolumeRegion object
        that represents the bounding region of the tile.

        Parameters
        ----------
        tile : morecantile.Tile
            The tile object.
        limit_to : list of float
            Optional list of west, south, east, north coordinates to limit
            the bounding region to.

        Returns
        -------
        bv : BoundingVolumeRegion
            The bounding region object.
        """
        tms = self.tiles.tms
        bounds = tms.bounds(tile)
        bounds = gpd.GeoSeries(
            box(bounds.left, bounds.bottom, bounds.right, bounds.top),
            crs=tms.crs)
        if limit_to is not None:
            bounds_limitor = gpd.GeoSeries(
                box(limit_to[0], limit_to[1], limit_to[2], limit_to[3]),
                crs=tms.crs)
            bounds = bounds.intersection(bounds_limitor)
        bounds = bounds.to_crs(BoundingVolumeRegion.CESIUM_EPSG)
        bounds = bounds.total_bounds

        region_bv = {
            'west': bounds[0], 'south': bounds[1],
            'east': bounds[2], 'north': bounds[3],
        }
        return region_bv

def create_leaf_3dtiles(staged_paths, config, logging_dict=None):
    """
    Create a batch of leaf 3d tiles from staged vector tiles
    """
    from pdg_workflow import StagedTo3DConverter
    if logging_dict:
        import logging.config
        logging.config.dictConfig(logging_dict)
    converter3d = StagedTo3DConverter(config)
    tilesets = []
    for path in staged_paths:
        try:
            ces_tile, ces_tileset = converter3d.staged_to_3dtile(path)
            tilesets.append(ces_tileset)
        except Exception as e:
            logging.error(f'Error creating 3d tile from {path}')
            logging.error(e)
    return tilesets

def create_parent_3dtiles(tiles, config, limit_bv_to=None, logging_dict=None):
    """
    Create a batch of cesium 3d tileset parent files that point to child
    tilesets
    """
    from pdg_workflow import StagedTo3DConverter
    if logging_dict:
        import logging.config
        logging.config.dictConfig(logging_dict)
    logging.info(f'Creating parent 3d tiles for {len(tiles)} tiles')
    converter3d = StagedTo3DConverter(config)
    return converter3d.parent_3dtiles_from_children(tiles, limit_bv_to)


def make_batch(items, batch_size):
    """
    Create batches of a given size from a list of items.
    """
    return [items[i:i + batch_size] for i in range(0, len(items), batch_size)]

Define the config as an object for 3dtiling.

In [ ]:
workflow_config = {
  "deduplicate_clip_to_footprint": True, 
  "dir_input": "/home/jcohen/iwp_russia_subset_clipToFP_PR/iwp/", 
  "ext_input": ".shp",
  "ext_footprints": ".shp",
  "dir_footprints": "/home/jcohen/iwp_russia_subset_clipToFP_PR/footprints/", 
  "dir_staged": "staged/",
  "dir_geotiff": "geotiff/", 
  "dir_web_tiles": "web_tiles/", 
  "filename_staging_summary": "staging_summary.csv",
  "filename_rasterization_events": "raster_events.csv",
  "filename_rasters_summary": "raster_summary.csv",
  "filename_config": "config",
  "simplify_tolerance": 0.1,
  "tms_id": "WGS1984Quad",
  "z_range": [
    0,
    15
  ],
  "geometricError": 57,
  "z_coord": 0,
  "statistics": [
    {
      "name": "iwp_coverage",
      "weight_by": "area",
      "property": "area_per_pixel_area",
      "aggregation_method": "sum",
      "resampling_method": "average",
      "val_range": [
        0,
        1
      ],
      "palette": [
        "#66339952",
        "#ffcc00"
      ],
      "nodata_val": 0,
      "nodata_color": "#ffffff00"
    },
  ],
  "deduplicate_at": [
    "raster"
  ],
  "deduplicate_keep_rules": [
    [
      "Date",
      "larger"
    ]
  ],
  "deduplicate_method": "footprints"
}

In [ ]:
# define batch sizes for this step
batch_size_3dtiles = 20
batch_size_parent_3dtiles = 500

stager = pdgstaging.TileStager(workflow_config)
tiles3dmaker = StagedTo3DConverter(workflow_config)
tile_manager = stager.tiles
config_manager = stager.config
min_z = config_manager.get_min_z()
max_z = config_manager.get_max_z()
parent_zs = range(max_z - 1, min_z - 1, -1)

In [ ]:
# Get paths to all the newly staged tiles
staged_paths = stager.tiles.get_filenames_from_dir('staged')
staged_batches = make_batch(staged_paths, batch_size_3dtiles)

for batch in staged_batches:
    create_leaf_3dtiles(batch, workflow_config) # no logging dict setup